In [1]:
# import libraray
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import pandas as pd
import numpy as np

In [2]:
# Define custom Dataset
class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __getitem__(self, index):
        features = self.X[index]
        target = self.y[index]
        return features, target

    def __len__(self):
        return len(self.X)

In [3]:
# Read Dataset
df = pd.read_csv("./dataframe.csv")
df = df.dropna(subset=['Year'])

X = df.iloc[:, 3:-1].values
y = df.iloc[:, -1].values

In [4]:
X = torch.from_numpy(X).type(torch.float32)
y = torch.from_numpy(y).type(torch.float32)
print(X.dtype)
print(y.dtype)

torch.float32
torch.float32


In [5]:
from sklearn.model_selection import train_test_split

# Assuming that X is your feature matrix and y are your targets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [6]:

# # # Import Dataset from preprocessing
# dataset = MyDataset(X, y)

# # Splitting dataset
# train_size = int(0.9 * len(dataset))
# val_size = len(dataset) - train_size
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [7]:
# # Instantiate a Dataloader with dataset
# # Create DataLoaders for the training and validation sets
# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [8]:
# Define the model

class MyRatingPredictionModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(MyRatingPredictionModel, self).__init__()
        self.linear1 = nn.Linear(input_size, 256)
        self.linear2 = nn.Linear(256, 256)
        self.relu = nn.ReLU()
        self.linear3 = nn.Linear(256, output_size)
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.linear3(x)
        return x

In [ ]:
# Instantiate and training the model
model = MyRatingPredictionModel(input_size = 20, output_size = 1)
# Define a loss function
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
# Training Loop
num_epochs = 100
for epoch in range(num_epochs):
    # Training loop
    model.train()  # Set the model to training mode
    
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient calculation
        total = 0
        correct = 0
       
        # Forward pass
        outputs = model(X_test)

        # Compute validation loss
        val_loss = criterion(outputs, y_test)

        # Compute validation accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += y_test.size(0)
        correct += (predicted == y_test).sum().item()

    print(f'Validation Loss: {val_loss.item()}, Validation Accuracy: {100 * correct / total}%')

D:\Anaconda\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([8757])) that is different to the input size (torch.Size([8757, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/100], Loss: 12.148222923278809
Validation Loss: 11.789937019348145, Validation Accuracy: 0.20554984583761562%


D:\Anaconda\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([973])) that is different to the input size (torch.Size([973, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [2/100], Loss: 12.03026294708252
Validation Loss: 11.675164222717285, Validation Accuracy: 0.20554984583761562%
Epoch [3/100], Loss: 11.914223670959473
Validation Loss: 11.56228256225586, Validation Accuracy: 0.20554984583761562%
Epoch [4/100], Loss: 11.800084114074707
Validation Loss: 11.451146125793457, Validation Accuracy: 0.20554984583761562%
Epoch [5/100], Loss: 11.687705039978027
Validation Loss: 11.341700553894043, Validation Accuracy: 0.20554984583761562%
Epoch [6/100], Loss: 11.577016830444336
Validation Loss: 11.233867645263672, Validation Accuracy: 0.20554984583761562%
Epoch [7/100], Loss: 11.467951774597168
Validation Loss: 11.127524375915527, Validation Accuracy: 0.20554984583761562%
Epoch [8/100], Loss: 11.360376358032227
Validation Loss: 11.02255630493164, Validation Accuracy: 0.20554984583761562%
Epoch [9/100], Loss: 11.254180908203125
Validation Loss: 10.918871879577637, Validation Accuracy: 0.20554984583761562%
Epoch [10/100], Loss: 11.149279594421387
Validation

Epoch [72/100], Loss: 5.588432788848877
Validation Loss: 5.353982448577881, Validation Accuracy: 0.20554984583761562%
Epoch [73/100], Loss: 5.504433631896973
Validation Loss: 5.271790981292725, Validation Accuracy: 0.20554984583761562%
Epoch [74/100], Loss: 5.420723915100098
Validation Loss: 5.189908027648926, Validation Accuracy: 0.20554984583761562%
Epoch [75/100], Loss: 5.337319850921631
Validation Loss: 5.108361721038818, Validation Accuracy: 0.20554984583761562%


In [10]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted)
print(f"MAE Score : {mae}")

MAE Score : 3.220695384604835
